<a href="https://colab.research.google.com/github/eshafayyazz/Quarter02/blob/main/Assignment05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --quiet google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 6.5 MB/s eta 0:00:00


In [ ]:
!pip install gTTS --quiet

In [ ]:
!pip install playsound --quiet

In [ ]:
from google.colab import userdata
GEMINI_API_KEY : str = userdata.get('Video_process')
if(GEMINI_API_KEY):
  print("API Key Found")
else:
  print("API Key Not Found")

API Key Found


In [ ]:
from google import genai
from google.genai import Client
Client : Client = genai.Client(api_key = GEMINI_API_KEY)
model= "gemini-2.0-flash-exp"

In [ ]:
video_file_name = "/content/introduction.mp4"

In [ ]:
import time

def upload_video(video_file_name):
  video_file = Client.files.upload(path=video_file_name)
  while video_file.state == "PROCESSING":
     print("Waiting for video to be processesd.")
     time.sleep(10)
     video_file = Client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
   raise ValueError(video_file.state)
  print(f'Video processing complete: ' + (video_file.uri or ""))

  return video_file

my_video = upload_video(video_file_name)


Waiting for video to be processesd.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/xb8ql1v3itxs


In [ ]:
from google.genai.types import Content,Part
prompt = """ For each scene in this video ,
             generate captions that describe the scene along with any spoken text placed in quotation marks.
             Place each caption into an object with the timecode of the caption in the video."""
video = my_video

response = Client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
            ]),
        prompt,
    ]
)
print(response.text)

```json
[
  {
    "timecode": "00:00",
    "caption": "A close up shot of a hand with the thumb extended forward, placed on a wooden surface. The spoken text is, \"Hello, my name is Esha Fiaz, and I am a student of computer science and exploring opportunities in artificial intelligence and app development.\""
  }
]
```


In [ ]:
from gtts import gTTS
from playsound import playsound
from IPython.display import Audio
tts = gTTS(text=response.text, lang='en')
with open("output.mp3", "wb") as f:
  for chunk in tts.stream():
    f.write(chunk)
display(Audio("output.mp3", autoplay=True))